In [ ]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
datasets = {
    "Eliot": 53, # Iris
    "Ethel": 728, # 
    "Leo": 878, # Cirrhosis
    "Adair": 109, # Wine
}


In [ ]:
dataset = fetch_ucirepo(id=datasets["Eliot"])
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 
df = pd.DataFrame(X, columns=dataset.data.feature_names)
df['target'] = y

In [ ]:
df = df.dropna()

In [ ]:
def separate_dataframe(df):
    vector_x = df.drop(df.columns[-1], axis=1) 
    vector_y = df[df.columns[-1]]
    return vector_x, vector_y

In [ ]:
vector_x, vector_y = separate_dataframe(df)

In [ ]:
vector_x = vector_x.select_dtypes(include=['int64', 'float64'])

In [ ]:
vector_x.head()

In [ ]:
vector_y.head()

----
# MODELOS

In [ ]:
def calcular_centroides(X, y):
    clases = set(y)
    centroides = {c: [0] * len(X[0]) for c in clases}
    contador = {c: 0 for c in clases}

    for xi, yi in zip(X, y):
        contador[yi] += 1
        for i in range(len(xi)):
            centroides[yi][i] += xi[i]

    for c in centroides:
        centroides[c] = [x / contador[c] for x in centroides[c]]
    
    return centroides

def clasificador_minima_distancia(X, centroides):
    predicciones = []
    for xi in X:
        distancias = {c: sum((xi[j] - centroides[c][j])**2 for j in range(len(xi))) for c in centroides}
        predicciones.append(min(distancias, key=distancias.get))
    return predicciones



In [ ]:
def clasificador_knn(X_train, y_train, X_test):
    predicciones = []
    for xi in X_test:
        distancias = [sum((xi[j] - X_train[i][j])**2 for j in range(len(xi))) for i in range(len(X_train))]
        min_index = distancias.index(min(distancias))
        predicciones.append(y_train[min_index])
    return predicciones

----
# METODOS DE VALIDACIÓN

- ### METODO TRAIN-TEST SPLIT

In [ ]:
def entrenamiento_prueba(X, y, porcentaje_prueba):
    indice_corte = int(len(X) * (1 - porcentaje_prueba))
    X_train = X[:indice_corte]
    y_train = y[:indice_corte]
    X_test = X[indice_corte:]
    y_test = y[indice_corte:]
    return X_train, y_train, X_test, y_test

- ### K FOLD CROSS VALIDATION

In [ ]:
def k_fold_cross_validation(X, y, k):
    tamaño_fold = len(X) // k
    for i in range(k):
        inicio = i * tamaño_fold
        fin = (i + 1) * tamaño_fold if i != k - 1 else len(X)
        X_train = X[:inicio] + X[fin:]
        y_train = y[:inicio] + y[fin:]
        X_test = X[inicio:fin]
        y_test = y[inicio:fin]
        yield X_train, y_train, X_test, y_test

- ### BOOTSTRAP

In [ ]:
import random

def bootstrap(X, y, n):
    for _ in range(n):
        indices = [random.randint(0, len(X) - 1) for _ in range(len(X))]
        X_train = [X[i] for i in indices]
        y_train = [y[i] for i in indices]
        X_test = [X[i] for i in range(len(X)) if i not in indices]
        y_test = [y[i] for i in range(len(X)) if i not in indices]
        yield X_train, y_train, X_test, y_test



In [ ]:
def calcular_eficiencia_error(y_real, y_pred):
    correctos = sum(1 for real, pred in zip(y_real, y_pred) if real == pred)
    eficiencia = correctos / len(y_real)
    error = 1 - eficiencia
    return eficiencia, error


-----
# USO DE LOS MODELOS

In [ ]:
def transformar_datos(vector_x, vector_y):
    X = vector_x.values.tolist() 
    # Transformar vector_y a una lista
    y = vector_y.values.tolist() 
    y = [item[0] for item in y] if isinstance(y[0], list) else y
    return X, y


- ## 4 => MODELO: DISTANCIA MINIMA
- ### 4.A => DISTANCIA MINIMA - EVALUADO CON: TRAIN-TEST SPLIT
- ### 4.B => DISTANCIA MINIMA - EVALUADO CON: K FOLD CROSS VALIDATION
- ### 4.C => DISTANCIA MINIMA - EVALUADO CON: BOOTSTRAP

In [ ]:
def run_min_distance_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Calcular centroides y hacer predicciones
    centroides = calcular_centroides(X_train, y_train)
    predicciones = clasificador_minima_distancia(X_test, centroides)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"Minima distancia: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 100  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        centroides = calcular_centroides(X_train, y_train)
        predicciones = clasificador_minima_distancia(X_test, centroides)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"Minima distancia: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")      



- ## 5 => MODELO: KNN(K=1)
- ### 5.A => KNN - EVALUADO CON: TRAIN-TEST SPLIT
- ### 5.B => KNN - EVALUADO CON: K FOLD CROSS VALIDATION
- ### 5.C => KNN - EVALUADO CON: BOOTSTRAP

In [ ]:
def run_knn_model(X, y):
    # ENTRENAMIENTO Y PRUEBA
    # Dividir los datos
    X_train, y_train, X_test, y_test = entrenamiento_prueba(X, y, porcentaje_prueba=0.2)
    # Hacer predicciones
    predicciones = clasificador_knn(X_train, y_train, X_test)
    # Calcular eficiencia y error
    eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
    print(f"KNN: eficiencia = {eficiencia}, error = {error} - Metodo de entrenamiento y prueba")
    
    # K-FOLD CROSS VALIDATION
    eficiencias = []
    errores = []
    k = 5  # Número de folds
    for X_train, y_train, X_test, y_test in k_fold_cross_validation(X, y, k):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / k
    error_promedio = sum(errores) / k
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - K-fold cross validation")
    
    # BOOTSTRAP
    eficiencias = []
    errores = []
    n_iteraciones = 100  # Número de iteraciones de Bootstrap
    for X_train, y_train, X_test, y_test in bootstrap(X, y, n_iteraciones):
        predicciones = clasificador_knn(X_train, y_train, X_test)
        eficiencia, error = calcular_eficiencia_error(y_test, predicciones)
        eficiencias.append(eficiencia)
        errores.append(error)
    # Calcular promedios
    eficiencia_promedio = sum(eficiencias) / n_iteraciones
    error_promedio = sum(errores) / n_iteraciones
    print(f"KNN: eficiencia = {eficiencia_promedio}, error = {error_promedio} - Bootstrap")       

In [ ]:
X, y = transformar_datos(vector_x, vector_y)

In [ ]:
# ejecutamos el modelo de distancia minima y validamos internamente con cada metodo
run_min_distance_model(X, y)

In [ ]:
# ejecutamos el modelo de KNN y validamos internamente con cada metodo
run_knn_model(X, y)